# Decade of CBSA level one-year estimates.

### *Imports*

In [1]:
import requests #for making API call
import json #for parsing API data
import pandas as pd #for formatting, cleaning, and exporting dataframe
import geopandas as gpd
pd.set_option('display.max_columns', None) #remove maximum on columns to display

In [2]:
#set API key variable
api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'

### Craft API calls for the population and home value data from the Census Bureau ACS 1 year estimates.

The purpose of this analysis is to find peer communities in regards to our aging population. The Census Bureau has data on age and population data outlined here: `https://api.census.gov/data/2019/acs/acs1/variables.html` that comes broken up by either sex, race, or ethnicity. There is a limit of 50 variables so the first call brings in total population and all male age groups, while the second brings in female age groups and home value.

In [3]:
#male 2010-2019 and total

#set the range of years as a list of strings to use .format()
range = ['10','11','12','13','14','15','16','17','18','19']

male_appended = [] #create empty list
for i in range:
    url_str= "https://api.census.gov/data/20{}/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key=".format(i)+api_key
    predicates= {}
    get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "state:*"
    male= requests.get(url_str, params= predicates)
    col_names = ['CBSA','GEOID','total','mtotal','mu5',
                 'm5_9','m10_14','m15_17','m18_19','m20',
                 'm21','m22_24','m25_29','m30_34',
                 'm35_39','m40_44','m45_49','m50_54','m55_59',
                 'm60_61','m62_64','m65_66','m67_69',
                 'm70_74','m75_79','m80_84','m85+','ind']
    male=pd.DataFrame(columns=col_names, data=male.json()[1:])
    male['year'] = '20{}'.format(i) #add a column with the year so that we have some tidydata
    male_appended.append(male)
    
male_appended = pd.concat(male_appended)
male = male_appended
print('Your API call is completed, congrats to you. You now have a dataframe called *male*.')

Your API call is completed, congrats to you. You now have a dataframe called *male*.


In [4]:
#check that you correctly imported each year
male.year.unique()

array(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019'], dtype=object)

In [5]:
male.head(3)

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind,year
0,"Aberdeen, WA Micro Area",310M100US10140,72882,37419,2177,1697,2600,1333,871,327,723,1399,2214,2269,2184,2680,2666,3104,3242,941,1440,653,936,1715,974,735,539,10140,2010
1,"Abilene, TX Metro Area",310M100US10180,164941,82694,5941,5059,5579,2942,3269,1390,1111,5685,6758,5375,4275,6111,5364,5418,5652,1426,2337,1716,1585,1775,1426,1204,1296,10180,2010
2,"Adrian, MI Micro Area",310M100US10300,99763,50213,2898,3360,3441,2211,1623,637,516,1905,3020,3153,3422,3175,4064,3838,4247,1150,1293,728,1137,1829,1059,844,663,10300,2010


In [6]:
male.tail()

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind,year
513,"York-Hanover, PA Metro Area",310M500US49620,449058,222750,13064,14438,14519,9764,5346,2914,2198,8070,14203,14259,13685,13455,14214,15332,14759,6626,9564,5669,7306,9448,6222,4891,2804,49620,2019
514,"Youngstown-Warren-Boardman, OH-PA Metro Area",310M500US49660,536081,262932,15166,14144,16198,10209,7238,3395,4444,9692,16109,14445,12708,16686,15671,17498,17892,8484,12154,7112,10189,12899,8975,5760,5864,49660,2019
515,"Yuba City, CA Metro Area",310M500US49700,175639,88746,6388,7803,5738,4123,2731,594,797,4346,7570,5304,6318,4890,4431,5377,4428,2713,3569,1845,2029,3320,1849,1173,1410,49700,2019
516,"Yuma, AZ Metro Area",310M500US49740,213787,110189,7557,7491,7850,4188,3236,1199,1618,7454,8713,7797,6839,5594,5058,5675,4680,2555,3364,1916,2652,4670,4793,3612,1678,49740,2019
517,"Zanesville, OH Micro Area",310M500US49780,86215,42589,2462,2046,3512,1527,1027,862,1850,1040,2734,2226,2081,2618,3000,2881,3677,944,1053,867,1349,2178,1328,608,719,49780,2019


In [7]:
male.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5203 entries, 0 to 517
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CBSA    5203 non-null   object
 1   GEOID   5203 non-null   object
 2   total   5203 non-null   object
 3   mtotal  5203 non-null   object
 4   mu5     5203 non-null   object
 5   m5_9    5203 non-null   object
 6   m10_14  5203 non-null   object
 7   m15_17  5203 non-null   object
 8   m18_19  5203 non-null   object
 9   m20     5203 non-null   object
 10  m21     5203 non-null   object
 11  m22_24  5203 non-null   object
 12  m25_29  5203 non-null   object
 13  m30_34  5203 non-null   object
 14  m35_39  5203 non-null   object
 15  m40_44  5203 non-null   object
 16  m45_49  5203 non-null   object
 17  m50_54  5203 non-null   object
 18  m55_59  5203 non-null   object
 19  m60_61  5203 non-null   object
 20  m62_64  5203 non-null   object
 21  m65_66  5203 non-null   object
 22  m67_69  5203 non-null   o

In [8]:
#although they won't each be in every year, curious about how many unique geographies we're dealing with
geo = male.CBSA.unique()
print(geo.size)

686


In [9]:
#look at one year just to see that our importing method is sound
twentynineteen = male.loc[male['year'] == '2019'].reset_index(drop = True)

In [10]:
twentynineteen.head(3)

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind,year
0,"Aberdeen, WA Micro Area",310M500US10140,75061,37879,1619,1749,2900,1006,278,659,402,1075,2213,2554,1767,2351,2576,2837,3118,983,1840,1030,1433,2606,1488,779,616,10140,2019
1,"Abilene, TX Metro Area",310M500US10180,171795,86747,5846,5253,6290,3630,2717,1861,1482,5169,7001,6303,5812,6603,3902,4210,5078,2140,2063,1859,2328,2317,2244,1219,1420,10180,2019
2,"Adrian, MI Micro Area",310M500US10300,98451,49744,2675,2573,2947,2112,1262,1009,572,1908,2633,3076,3292,3107,3254,3232,3804,1453,2012,756,2249,2513,1991,715,599,10300,2019


In [11]:
twentynineteen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CBSA    518 non-null    object
 1   GEOID   518 non-null    object
 2   total   518 non-null    object
 3   mtotal  518 non-null    object
 4   mu5     518 non-null    object
 5   m5_9    518 non-null    object
 6   m10_14  518 non-null    object
 7   m15_17  518 non-null    object
 8   m18_19  518 non-null    object
 9   m20     518 non-null    object
 10  m21     518 non-null    object
 11  m22_24  518 non-null    object
 12  m25_29  518 non-null    object
 13  m30_34  518 non-null    object
 14  m35_39  518 non-null    object
 15  m40_44  518 non-null    object
 16  m45_49  518 non-null    object
 17  m50_54  518 non-null    object
 18  m55_59  518 non-null    object
 19  m60_61  518 non-null    object
 20  m62_64  518 non-null    object
 21  m65_66  518 non-null    object
 22  m67_69  518 non-null    ob

Craft an API call to only bring in 2019 and compare the two.

In [12]:
#male 2019 and total
api_key = '24fc7d81b74510d599f702dbd408fb18e1466d81'
url_str= 'https://api.census.gov/data/2019/acs/acs1?get=NAME,GEO_ID,B01001_001E,B01001_002E,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_013E,B01001_014E,B01001_015E,B01001_016E,B01001_017E,B01001_018E,B01001_019E,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='+api_key

predicates= {}
get_vars= ["NAME","GEO_ID",'B01001_001E','B01001_020E','B01001_003E','B01001_004E','B01001_005E','B01001_006E','B01001_007E','B01001_008E','B01001_009E','B01001_010E','B01001_011E','B01001_012E','B01001_013E','B01001_014E','B01001_015E','B01001_016E','B01001_017E','B01001_018E','B01001_019E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:*"
male2019= requests.get(url_str, params= predicates)

col_names = ['CBSA','GEOID','total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
            'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79','m80_84','m85+','ind']
male2019=pd.DataFrame(columns=col_names, data=male2019.json()[1:])

In [13]:
male2019.head(3)

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind
0,"Aberdeen, WA Micro Area",310M500US10140,75061,37879,1619,1749,2900,1006,278,659,402,1075,2213,2554,1767,2351,2576,2837,3118,983,1840,1030,1433,2606,1488,779,616,10140
1,"Abilene, TX Metro Area",310M500US10180,171795,86747,5846,5253,6290,3630,2717,1861,1482,5169,7001,6303,5812,6603,3902,4210,5078,2140,2063,1859,2328,2317,2244,1219,1420,10180
2,"Adrian, MI Micro Area",310M500US10300,98451,49744,2675,2573,2947,2112,1262,1009,572,1908,2633,3076,3292,3107,3254,3232,3804,1453,2012,756,2249,2513,1991,715,599,10300


In [14]:
male2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CBSA    518 non-null    object
 1   GEOID   518 non-null    object
 2   total   518 non-null    object
 3   mtotal  518 non-null    object
 4   mu5     518 non-null    object
 5   m5_9    518 non-null    object
 6   m10_14  518 non-null    object
 7   m15_17  518 non-null    object
 8   m18_19  518 non-null    object
 9   m20     518 non-null    object
 10  m21     518 non-null    object
 11  m22_24  518 non-null    object
 12  m25_29  518 non-null    object
 13  m30_34  518 non-null    object
 14  m35_39  518 non-null    object
 15  m40_44  518 non-null    object
 16  m45_49  518 non-null    object
 17  m50_54  518 non-null    object
 18  m55_59  518 non-null    object
 19  m60_61  518 non-null    object
 20  m62_64  518 non-null    object
 21  m65_66  518 non-null    object
 22  m67_69  518 non-null    ob

Looks good to me. Now the call for female and home value...

In [15]:
#female 2010-2019 and home value

#set the range of years as a list of strings to use .format()
range = ['10','11','12','13','14','15','16','17','18','19']
female_appended = [] #create empty list
for i in range:
    url_str= 'https://api.census.gov/data/20{}/acs/acs1?get=NAME,GEO_ID,B25077_001E,B01001_026E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_037E,B01001_038E,B01001_039E,B01001_040E,B01001_041E,B01001_042E,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key='.format(i)+api_key
    predicates= {}
    get_vars= ["NAME","GEO_ID",'B25077_001E','B01001_026E','B01001_027E','B01001_028E','B01001_029E','B01001_030E','B01001_031E','B01001_032E','B01001_033E','B01001_034E','B01001_035E','B01001_036E','B01001_037E','B01001_038E','B01001_039E','B01001_040E','B01001_041E','B01001_042E','B01001_043E','B01001_044E','B01001_045E','B01001_046E','B01001_047E','B01001_048E','B01001_049E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "state:*"
    female2019= requests.get(url_str, params= predicates)

    col_names = ['CBSA','GEOID','homevalue','ftotal','fu5','f5_9','f10_14','f15_17','f18_19','f20','f21','f22_24','f25_29','f30_34',
                'f35_39','f40_44','f45_49','f50_54','f55_59','f60_61','f62_64','f65_66','f67_69','f70_74','f75_79','f80_84','f85+','ind']
    female=pd.DataFrame(columns=col_names, data=female2019.json()[1:])
    female['year'] = '20{}'.format(i) #add a column with the year so that we have some tidydata
    female_appended.append(female)
    
female_appended = pd.concat(female_appended)
female = female_appended
print('Your API call is completed, congrats to you. You now have a dataframe called *female*.')

Your API call is completed, congrats to you. You now have a dataframe called *female*.


In [16]:
female.head(3)

,CBSA,GEOID,homevalue,ftotal,fu5,f5_9,f10_14,f15_17,f18_19,f20,f21,f22_24,f25_29,f30_34,f35_39,f40_44,f45_49,f50_54,f55_59,f60_61,f62_64,f65_66,f67_69,f70_74,f75_79,f80_84,f85+,ind,year
0,"Aberdeen, WA Micro Area",310M100US10140,162200,35463,1903,2719,1584,1329,674,317,424,1719,1989,1839,2105,1704,2477,2854,3012,1402,1310,337,1103,1728,1505,818,611,10140,2010
1,"Abilene, TX Metro Area",310M100US10180,90900,82247,5603,4599,6074,2948,2916,1875,1540,4023,5660,4273,4738,5005,4981,5314,5064,2079,2500,1559,2069,2659,3125,1190,2453,10180,2010
2,"Adrian, MI Micro Area",310M100US10300,118000,49550,2713,3286,3513,1768,1405,949,277,1593,2239,2907,3001,3354,3556,3660,3936,1094,1746,838,2101,1523,1571,1171,1349,10300,2010


We need a unique identifier to join this data. That can be accomplished by concatenating the GEOID and the year.

In [17]:
male['geoyearID']=male['GEOID']+male['year']
female['geoyearID']=female['GEOID']+female['year']

In [18]:
#make sure the datatypes are correct
male['geoyearID'] = male['geoyearID'].astype(str)
female['geoyearID'] = female['geoyearID'].astype(str)

In [19]:
data = pd.merge(male, female, on='geoyearID', how = 'inner')

Check out our joined dataframe.

In [20]:
data.head()

,CBSA_x,GEOID_x,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,ind_x,year_x,geoyearID,CBSA_y,GEOID_y,homevalue,ftotal,fu5,f5_9,f10_14,f15_17,f18_19,f20,f21,f22_24,f25_29,f30_34,f35_39,f40_44,f45_49,f50_54,f55_59,f60_61,f62_64,f65_66,f67_69,f70_74,f75_79,f80_84,f85+,ind_y,year_y
0,"Aberdeen, WA Micro Area",310M100US10140,72882,37419,2177,1697,2600,1333,871,327,723,1399,2214,2269,2184,2680,2666,3104,3242,941,1440,653,936,1715,974,735,539,10140,2010,310M100US101402010,"Aberdeen, WA Micro Area",310M100US10140,162200,35463,1903,2719,1584,1329,674,317,424,1719,1989,1839,2105,1704,2477,2854,3012,1402,1310,337,1103,1728,1505,818,611,10140,2010
1,"Abilene, TX Metro Area",310M100US10180,164941,82694,5941,5059,5579,2942,3269,1390,1111,5685,6758,5375,4275,6111,5364,5418,5652,1426,2337,1716,1585,1775,1426,1204,1296,10180,2010,310M100US101802010,"Abilene, TX Metro Area",310M100US10180,90900,82247,5603,4599,6074,2948,2916,1875,1540,4023,5660,4273,4738,5005,4981,5314,5064,2079,2500,1559,2069,2659,3125,1190,2453,10180,2010
2,"Adrian, MI Micro Area",310M100US10300,99763,50213,2898,3360,3441,2211,1623,637,516,1905,3020,3153,3422,3175,4064,3838,4247,1150,1293,728,1137,1829,1059,844,663,10300,2010,310M100US103002010,"Adrian, MI Micro Area",310M100US10300,118000,49550,2713,3286,3513,1768,1405,949,277,1593,2239,2907,3001,3354,3556,3660,3936,1094,1746,838,2101,1523,1571,1171,1349,10300,2010
3,"Aguadilla-Isabela-San Sebasti?n, PR Metro Area",310M100US10380,305988,149560,9529,10792,10016,6552,5445,2185,1854,5578,9276,9324,10716,9875,9928,9641,8462,4236,5309,3520,4598,5104,3160,2670,1790,10380,2010,310M100US103802010,"Aguadilla-Isabela-San Sebasti?n, PR Metro Area",310M100US10380,97400,156428,8358,9611,10102,7638,4564,2692,1715,6307,8480,10120,11297,10267,10764,10737,9675,4169,6559,3311,5115,5400,3815,2718,3014,10380,2010
4,"Akron, OH Metro Area",310M100US10420,702951,341630,20054,20805,24467,15115,11518,4644,5620,15959,20638,19911,19937,22907,25878,27269,24857,8941,10447,5069,8071,10656,8882,6239,3746,10420,2010,310M100US104202010,"Akron, OH Metro Area",310M100US10420,145000,361321,19117,20458,22134,14435,11638,5989,5456,13883,21706,20308,20670,24553,26953,28762,25904,8254,13138,6093,9551,12251,11052,9374,9642,10420,2010


Drop the indices and extra CBSA name column, rename the one you're going to keep, then convert all numeric columns to int64 as we're going to be performing mathematical operations on them.

In [21]:
#drop the indices and extra state name
data = data.drop(columns=['ind_x','year_x','CBSA_y','GEOID_y','ind_y'])

In [22]:
data = data.rename(columns={'CBSA_x':'CBSA','GEOID_x':'GEOID', 'year_y':'year'})

In [23]:
#convert all numeric columns to integers
cols = ['total','mtotal','mu5','m5_9','m10_14','m15_17','m18_19','m20','m21','m22_24','m25_29','m30_34',
        'm35_39','m40_44','m45_49','m50_54','m55_59','m60_61','m62_64','m65_66','m67_69','m70_74','m75_79',
        'm80_84','m85+','ftotal', 'fu5','f5_9','f10_14','f15_17','f18_19','f20','f21','f22_24','f25_29',
        'f30_34','f35_39','f40_44','f45_49','f50_54','f55_59','f60_61','f62_64','f65_66','f67_69','f70_74',
        'f75_79','f80_84','f85+','homevalue']

data[cols]=data[cols].apply(pd.to_numeric, errors='coerce', axis=1)

Check that the last few steps were effective.

In [24]:
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5203 entries, 0 to 5202
Data columns (total 54 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CBSA       5203 non-null   object
 1   GEOID      5203 non-null   object
 2   total      5203 non-null   int64 
 3   mtotal     5203 non-null   int64 
 4   mu5        5203 non-null   int64 
 5   m5_9       5203 non-null   int64 
 6   m10_14     5203 non-null   int64 
 7   m15_17     5203 non-null   int64 
 8   m18_19     5203 non-null   int64 
 9   m20        5203 non-null   int64 
 10  m21        5203 non-null   int64 
 11  m22_24     5203 non-null   int64 
 12  m25_29     5203 non-null   int64 
 13  m30_34     5203 non-null   int64 
 14  m35_39     5203 non-null   int64 
 15  m40_44     5203 non-null   int64 
 16  m45_49     5203 non-null   int64 
 17  m50_54     5203 non-null   int64 
 18  m55_59     5203 non-null   int64 
 19  m60_61     5203 non-null   int64 
 20  m62_64     5203 non-null   int

Create columns of totals for each one of these age groups. Each row is created by joining on the unique geoyearID so these totals should be correct.

In [25]:
#create totals columns for "new":
data['tu5'] = data['mu5']+data['fu5']
data['t5_9'] = data['m5_9']+data['f5_9']
data['t10_14'] = data['m10_14']+data['f10_14']
data['t15_17'] = data['m15_17']+data['f15_17']
data['t18_19'] = data['m18_19']+data['f18_19']
data['t20'] = data['m20']+data['f20']
data['t21'] = data['m21']+data['f21']
data['t22_24'] = data['m22_24']+data['f22_24']
data['t25_29'] = data['m25_29']+data['f25_29']
data['t30_34'] = data['m30_34']+data['f30_34']
data['t35_39'] = data['m35_39']+data['f35_39']
data['t40_44'] = data['m40_44']+data['f40_44']
data['t45_49'] = data['m45_49']+data['f45_49']
data['t50_54'] = data['m50_54']+data['f50_54']
data['t55_59'] = data['m55_59']+data['f55_59']
data['t60_61'] = data['m60_61']+data['f60_61']
data['t62_64'] = data['m62_64']+data['f62_64']
data['t65_66'] = data['m65_66']+data['f65_66']
data['t67_69'] = data['m67_69']+data['f67_69']
data['t70_74'] = data['m70_74']+data['f70_74']
data['t75_79'] = data['m75_79']+data['f75_79']
data['t80_84'] = data['m80_84']+data['f80_84']
data['t85+'] = data['m85+']+data['f85+']

data = data.copy()

### Create groups for analysis. Since we're looking at ageing services, it makes sense to group people into some rough "stage of life" that is predicated on services provided to them or their position in the workforce.

###### So we're going to do:  
- under 18  
- 18 - 64  
- 65+  

Note that we're also eliminating the gender divide, we can always come back and disaggregate this but it doesn't seem particularly relevant now.

In [26]:
#child
data['child'] = data['tu5']+data['t5_9']+data['t10_14']+data['t15_17']
#taxbase
data['taxbase'] = data['t18_19']+data['t20']+data['t21']+data['t22_24']+data['t25_29']+data['t30_34']+data['t35_39']+data['t40_44']+data['t45_49']+data['t50_54']+data['t55_59']+data['t60_61']+data['t62_64']
#over 65
data['O65'] = data['t65_66']+data['t67_69']+data['t70_74']+data['t75_79']+data['t80_84']+data['t85+']

data = data.copy()

Make these groups into shares of the total.

In [27]:
data['Pchild']=round(data['child']*100/data['total'],1)
data['Ptaxbase']=round(data['taxbase']*100/data['total'],1)
data['PO65']=round(data['O65']*100/data['total'],1)

data = data.copy()

In [28]:
data.head(2)

,CBSA,GEOID,total,mtotal,mu5,m5_9,m10_14,m15_17,m18_19,m20,m21,m22_24,m25_29,m30_34,m35_39,m40_44,m45_49,m50_54,m55_59,m60_61,m62_64,m65_66,m67_69,m70_74,m75_79,m80_84,m85+,geoyearID,homevalue,ftotal,fu5,f5_9,f10_14,f15_17,f18_19,f20,f21,f22_24,f25_29,f30_34,f35_39,f40_44,f45_49,f50_54,f55_59,f60_61,f62_64,f65_66,f67_69,f70_74,f75_79,f80_84,f85+,year,tu5,t5_9,t10_14,t15_17,t18_19,t20,t21,t22_24,t25_29,t30_34,t35_39,t40_44,t45_49,t50_54,t55_59,t60_61,t62_64,t65_66,t67_69,t70_74,t75_79,t80_84,t85+,child,taxbase,O65,Pchild,Ptaxbase,PO65
0,"Aberdeen, WA Micro Area",310M100US10140,72882,37419,2177,1697,2600,1333,871,327,723,1399,2214,2269,2184,2680,2666,3104,3242,941,1440,653,936,1715,974,735,539,310M100US101402010,162200,35463,1903,2719,1584,1329,674,317,424,1719,1989,1839,2105,1704,2477,2854,3012,1402,1310,337,1103,1728,1505,818,611,2010,4080,4416,4184,2662,1545,644,1147,3118,4203,4108,4289,4384,5143,5958,6254,2343,2750,990,2039,3443,2479,1553,1150,15342,45886,11654,21.1,63.0,16.0
1,"Abilene, TX Metro Area",310M100US10180,164941,82694,5941,5059,5579,2942,3269,1390,1111,5685,6758,5375,4275,6111,5364,5418,5652,1426,2337,1716,1585,1775,1426,1204,1296,310M100US101802010,90900,82247,5603,4599,6074,2948,2916,1875,1540,4023,5660,4273,4738,5005,4981,5314,5064,2079,2500,1559,2069,2659,3125,1190,2453,2010,11544,9658,11653,5890,6185,3265,2651,9708,12418,9648,9013,11116,10345,10732,10716,3505,4837,3275,3654,4434,4551,2394,3749,38745,104139,22057,23.5,63.1,13.4


Make a separate dataframe of only these created groups and their shares

In [29]:
data = data[['CBSA', 'GEOID', 'year', 'geoyearID', 'total', 'child', 'taxbase', 'O65', 
           'Pchild', 'Ptaxbase', 'PO65', 'homevalue']]

In [30]:
data.head()

,CBSA,GEOID,year,geoyearID,total,child,taxbase,O65,Pchild,Ptaxbase,PO65,homevalue
0,"Aberdeen, WA Micro Area",310M100US10140,2010,310M100US101402010,72882,15342,45886,11654,21.1,63.0,16.0,162200
1,"Abilene, TX Metro Area",310M100US10180,2010,310M100US101802010,164941,38745,104139,22057,23.5,63.1,13.4,90900
2,"Adrian, MI Micro Area",310M100US10300,2010,310M100US103002010,99763,23190,61760,14813,23.2,61.9,14.8,118000
3,"Aguadilla-Isabela-San Sebasti?n, PR Metro Area",310M100US10380,2010,310M100US103802010,305988,72598,189175,44215,23.7,61.8,14.4,97400
4,"Akron, OH Metro Area",310M100US10420,2010,310M100US104202010,702951,156585,445740,100626,22.3,63.4,14.3,145000


Let's split the full GEOID so that we have the CBSA FIPS, let's also split up the CBSA names so that we have the name, state, and Metro or Micro Area designation. Start with the geography name.

In [31]:
# Split the CBSA column at the comma then name the new column as the first column you index into
name = data['CBSA'].str.split(pat = ',', expand = True)
name

,0,1,2
0,Aberdeen,WA Micro Area,None
1,Abilene,TX Metro Area,None
2,Adrian,MI Micro Area,None
3,Aguadilla-Isabela-San Sebasti?n,PR Metro Area,None
4,Akron,OH Metro Area,None
...,...,...,...
5198,York-Hanover,PA Metro Area,None
5199,Youngstown-Warren-Boardman,OH-PA Metro Area,None
5200,Yuba City,CA Metro Area,None
5201,Yuma,AZ Metro Area,None


In [32]:
data['Name'] = name[0]
state_area = name[1]
state_area

0           WA Micro Area
1           TX Metro Area
2           MI Micro Area
3           PR Metro Area
4           OH Metro Area
              ...        
5198        PA Metro Area
5199     OH-PA Metro Area
5200        CA Metro Area
5201        AZ Metro Area
5202        OH Micro Area
Name: 1, Length: 5203, dtype: object

In [33]:
# Next we can split this again after the first two figures which indicate the State
state = state_area.str.slice(stop = 3)
state

0        WA
1        TX
2        MI
3        PR
4        OH
       ... 
5198     PA
5199     OH
5200     CA
5201     AZ
5202     OH
Name: 1, Length: 5203, dtype: object

In [34]:
data['State'] = state

In [35]:
area = state_area.str.slice(start = 3, stop = 9)
area

0        Micro
1        Metro
2        Micro
3        Metro
4        Metro
         ...  
5198     Metro
5199    -PA Me
5200     Metro
5201     Metro
5202     Micro
Name: 1, Length: 5203, dtype: object

In [36]:
data['MetroMicro'] = area

Now we're going to split the GEOIDs.

In [37]:
data['CBSAFIPS'] = data['GEOID'].str.slice(start = 9)

In [38]:
data.head()

,CBSA,GEOID,year,geoyearID,total,child,taxbase,O65,Pchild,Ptaxbase,PO65,homevalue,Name,State,MetroMicro,CBSAFIPS
0,"Aberdeen, WA Micro Area",310M100US10140,2010,310M100US101402010,72882,15342,45886,11654,21.1,63.0,16.0,162200,Aberdeen,WA,Micro,10140
1,"Abilene, TX Metro Area",310M100US10180,2010,310M100US101802010,164941,38745,104139,22057,23.5,63.1,13.4,90900,Abilene,TX,Metro,10180
2,"Adrian, MI Micro Area",310M100US10300,2010,310M100US103002010,99763,23190,61760,14813,23.2,61.9,14.8,118000,Adrian,MI,Micro,10300
3,"Aguadilla-Isabela-San Sebasti?n, PR Metro Area",310M100US10380,2010,310M100US103802010,305988,72598,189175,44215,23.7,61.8,14.4,97400,Aguadilla-Isabela-San Sebasti?n,PR,Metro,10380
4,"Akron, OH Metro Area",310M100US10420,2010,310M100US104202010,702951,156585,445740,100626,22.3,63.4,14.3,145000,Akron,OH,Metro,10420


Let's bring in a shapefile to calculate different densities, but then drop the geometries etc. to save memory. They can be rejoined later to map if need be.

In [39]:
#extract GeoDataFrame from Tiger
url= 'https://www2.census.gov/geo/tiger/TIGER2019/CBSA/tl_2019_us_cbsa.zip'
geo = gpd.read_file(url)
geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 938 entries, 0 to 937
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   CSAFP     550 non-null    object  
 1   CBSAFP    938 non-null    object  
 2   GEOID     938 non-null    object  
 3   NAME      938 non-null    object  
 4   NAMELSAD  938 non-null    object  
 5   LSAD      938 non-null    object  
 6   MEMI      938 non-null    object  
 7   MTFCC     938 non-null    object  
 8   ALAND     938 non-null    int64   
 9   AWATER    938 non-null    int64   
 10  INTPTLAT  938 non-null    object  
 11  INTPTLON  938 non-null    object  
 12  geometry  938 non-null    geometry
dtypes: geometry(1), int64(2), object(10)
memory usage: 95.4+ KB


In [40]:
geo['CBSAFP'] = geo['CBSAFP'].astype(int)
data['CBSAFIPS']=data['CBSAFIPS'].astype(int)

In [41]:
data_geo = geo.merge(data, left_on='CBSAFP', right_on='CBSAFIPS')

###### Convert land and water areas to proper units.
We have this in square meters, so the converstion factor is 0.0000003861 to get this to square miles.

In [42]:
data_geo['ALAND'] = data_geo['ALAND'] * 0.0000003861
data_geo['AWATER'] = data_geo['AWATER'] * 0.0000003861

In [43]:
data_geo['popdensity']=data_geo['total']/data_geo['ALAND']
data_geo['O65popdensity']=data_geo['O65']/data_geo['ALAND']
data_geo['tbpopdensity']=data_geo['taxbase']/data_geo['ALAND']
data_geo['childpopdensity']=data_geo['child']/data_geo['ALAND']

In [44]:
data = data_geo

In [45]:
data.head(2)

,CSAFP,CBSAFP,GEOID_x,NAME,NAMELSAD,LSAD,MEMI,MTFCC,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,CBSA,GEOID_y,year,geoyearID,total,child,taxbase,O65,Pchild,Ptaxbase,PO65,homevalue,Name,State,MetroMicro,CBSAFIPS,popdensity,O65popdensity,tbpopdensity,childpopdensity
0,122,12020,12020,"Athens-Clarke County, GA","Athens-Clarke County, GA Metro Area",M1,1,G3110,1024.941767,10.092773,+33.9439840,-083.2138965,"POLYGON ((-83.53739 33.96591, -83.53184 33.968...","Athens-Clarke County, GA Metro Area",310M100US12020,2010,310M100US120202010,192738,40837,132080,19821,21.2,68.5,10.3,160200,Athens-Clarke County,GA,Metro,12020,188.047757,19.338660,128.865858,39.843239
1,122,12020,12020,"Athens-Clarke County, GA","Athens-Clarke County, GA Metro Area",M1,1,G3110,1024.941767,10.092773,+33.9439840,-083.2138965,"POLYGON ((-83.53739 33.96591, -83.53184 33.968...","Athens-Clarke County, GA Metro Area",310M100US12020,2011,310M100US120202011,193317,39216,133321,20780,20.3,69.0,10.7,163600,Athens-Clarke County,GA,Metro,12020,188.612667,20.274323,130.076658,38.261686


In [46]:
data = data.drop(columns=['CSAFP','CBSAFP','GEOID_x','NAMELSAD','LSAD','MEMI','Name','MTFCC','ALAND','AWATER','INTPTLAT','INTPTLON','geometry'])

In [47]:
data = data.rename(columns={'GEOID_y':'GEOID'})

In [54]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 5053 entries, 0 to 5052
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   NAME             5053 non-null   object 
 1   CBSA             5053 non-null   object 
 2   GEOID            5053 non-null   object 
 3   year             5053 non-null   object 
 4   geoyearID        5053 non-null   object 
 5   total            5053 non-null   int64  
 6   child            5053 non-null   int64  
 7   taxbase          5053 non-null   int64  
 8   O65              5053 non-null   int64  
 9   Pchild           5053 non-null   float64
 10  Ptaxbase         5053 non-null   float64
 11  PO65             5053 non-null   float64
 12  homevalue        5053 non-null   int64  
 13  State            5053 non-null   object 
 14  MetroMicro       5053 non-null   object 
 15  CBSAFIPS         5053 non-null   int32  
 16  popdensity       5053 non-null   float64
 17  O65pop

In [51]:
data.corr()

,total,child,taxbase,O65,Pchild,Ptaxbase,PO65,homevalue,CBSAFIPS,popdensity,O65popdensity,tbpopdensity,childpopdensity
total,1.000000,0.996006,0.999778,0.986582,0.073722,0.125371,-0.151202,0.307589,0.016524,0.728799,0.628409,0.737905,0.744832
child,0.996006,1.000000,0.995460,0.970833,0.111781,0.122652,-0.176227,0.292637,0.012515,0.706168,0.597277,0.715158,0.729464
taxbase,0.999778,0.995460,1.000000,0.984859,0.071295,0.133935,-0.156204,0.310880,0.015287,0.729123,0.626172,0.738904,0.745032
O65,0.986582,0.970833,0.984859,1.000000,0.019834,0.087740,-0.083155,0.310867,0.028573,0.749697,0.677438,0.755684,0.753842
Pchild,0.073722,0.111781,0.071295,0.019834,1.000000,-0.113507,-0.624396,-0.131499,0.035121,-0.059318,-0.189299,-0.057260,0.029314
Ptaxbase,0.125371,0.122652,0.133935,0.087740,-0.113507,1.000000,-0.705100,0.128421,-0.152047,0.101850,-0.045830,0.136034,0.105034
PO65,-0.151202,-0.176227,-0.156204,-0.083155,-0.624396,-0.705100,1.000000,-0.007002,0.094672,-0.037804,0.171071,-0.066147,-0.103564
homevalue,0.307589,0.292637,0.310880,0.310867,-0.131499,0.128421,-0.007002,1.000000,0.069213,0.362023,0.330915,0.371566,0.342386
CBSAFIPS,0.016524,0.012515,0.015287,0.028573,0.035121,-0.152047,0.094672,0.069213,1.000000,0.028453,0.041754,0.025594,0.026033
popdensity,0.728799,0.706168,0.729123,0.749697,-0.059318,0.101850,-0.037804,0.362023,0.028453,1.000000,0.959863,0.998753,0.991973


### Export

I'm going to export here and we can explore the data further in a new notebook - I want to separate prep for simplicity's sake.

In [55]:
data.to_csv('../output/csv/decadeCBSAs.csv', index = False)

In [56]:
kaggle competitions download -c store-sales-time-series-forecasting

SyntaxError: invalid syntax (Temp/ipykernel_22904/4214269598.py, line 1)